### Parameter MLPs
This notebook uses the `NNComponentsCV` model.

In [19]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter

import sys
import joblib
import datetime
sys.path.append('../')

from Modules.Utils.ModelWrapper import ModelWrapper
from Modules.Models.BuildBINNs import NNComponentsCV
from Modules.Utils.Imports import *

import Modules.Loaders.DataFormatter as DF
from utils import plot_loss_convergence, get_case_name
import matplotlib
# matplotlib.use('Agg')

In [20]:
device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))
# helper functions
def to_torch(ndarray):
    arr = torch.tensor(ndarray, dtype=torch.float)
    arr.requires_grad_(True)
    arr = arr.to(device)
    return arr

def to_numpy(x):
    return x.detach().cpu().numpy()

Device set to cuda:0


In [21]:
# instantiate BINN model parameters and path
path = '../Data/covasim_data/drums_data/'
population = int(200e3) # 200000, 100000
test_prob = 0.1
trace_prob = 0.3
keep_d = True
retrain = False
dynamic = True
masking = 3
parallelb = True
n_runs = 1024
chi_type = 'piecewise'

plot = False
trim = False

case_name = get_case_name(population, test_prob, trace_prob, keep_d, dynamic=dynamic, chi_type=chi_type)

In [22]:
if not masking==0:
    if masking==1:
        case_name = case_name + '_maskingthresh'
    elif masking==2:
        case_name = case_name + '_maskinguni'
    elif masking==3:
        case_name = case_name + '_maskingnorm'
    
params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name + '_' + str(n_runs), plot=False)

In [23]:
if parallelb:
    data_smooth = params['data']
else:
    data_smooth = params['data']
    data_smooth = np.mean(data_smooth, axis=1)
    data_smooth = (data_smooth / params['population'])
    
if trim:
    N = len(data_smooth) - 17
    u = to_torch(data_smooth[16:N+16,:])
else:
    N = len(data_smooth) - 2
    u = to_torch(data_smooth[1:N+1,:])

t_max_real = N + 1
t = np.arange(N)[:,None] + 1
params.pop('data')

if masking>0:
    avg_masking = params['avg_masking']
tracing_array = params['tracing_array']

In [24]:
dmethod = 'savgol' # 'savgol', 'cfd'
# central differences numerical differentiation
if dmethod=='cfd':
    if trim:
        u_front = data_smooth[:N+15,:]
        u_back = data_smooth[2:,:]
    else:
        u_front = data_smooth[:N,:]
        u_back = data_smooth[2:,:]
        
    ut = to_torch((u_back - u_front) / 2.)
    
elif dmethod=='savgol':
    ut = to_torch(savgol_filter(data_smooth, 9, 3, deriv=1, axis=0))[1:N+1,:]

In [25]:
# split into train/val and convert to torch
split = int(0.8*N)
# generate shuffled array of indices from 0 to N-1
p = np.random.permutation(N)[:,None]

if trim:
    u_tensor = torch.cat([u[:,:,None], ut[15:,:,None]], axis=2)
else:
    u_tensor = torch.cat([u[:,:,None], ut[:,:,None]], axis=2)

In [26]:
# assign x_train to be randomly shuffled days from 1 to N of size int(0.8 * N)
x_train = to_torch((p[:split] + 1)) / t_max_real
# assign y_train to be values corresponding to x_train of size int(0.8 * N)
y_train = to_torch(u[(p[:split]).flatten()])
# assign x_val to be randomly shuffled days from 1 to N of size int(0.2 * N)
x_val = to_torch((p[split:] + 1)) / t_max_real
# assign y_val to be values corresponding to y_val of size int(0.2 * N)
y_val = to_torch(u[(p[split:]).flatten()])

/home/mars/venv3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [27]:
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (8, 6)

if plot:
    start_idx = 0
    if trim:
        start_idx = 15
    dSdt = ut[start_idx:,0].detach()
    dTdt = ut[start_idx:,1].detach()
    dEdt = ut[start_idx:,2].detach()
    dAdt = ut[start_idx:,3].detach()
    dYdt = ut[start_idx:,4].detach()
    dDdt = ut[start_idx:,5].detach()
    dQdt = ut[start_idx:,6].detach()
    dRdt = ut[start_idx:,7].detach()
    dFdt = ut[start_idx:,8].detach()
    for i in range(9):
        if i==0:
            plt.title('$dSdt$ versus time (days)')
            plt.plot(t, dSdt*population, label='dSdt', color='b')
            plt.xlabel('Time (days)')
            plt.ylabel('$dSdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dSdt' + '.png')
            plt.show()
            plt.close()
        if i==1:
            plt.title('$dTdt$ versus time (days)')
            plt.plot(t, dTdt*population, label='dTdt', color='b')
            plt.xlabel('Time (days)')
            plt.ylabel('$dTdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dTdt' + '.png')
            plt.show()
            plt.close()
        if i==2:
            plt.title('$dEdt$ versus time (days)')
            plt.plot(t, dEdt*population, label='dEdt', color='y')
            plt.xlabel('Time (days)')
            plt.ylabel('$dEdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dEdt' + '.png')
            plt.show()
            plt.close()
        if i==3:
            plt.title('$dAdt$ versus time (days)')
            plt.plot(t, dAdt*population, label='dAdt', color='r')
            plt.xlabel('Time (days)')
            plt.ylabel('$dAdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dAdt' + '.png')
            plt.show()
            plt.close()
        if i==4:
            plt.title('$dYdt$ versus time (days)')
            plt.plot(t, dYdt*population, label='dYdt', color='r')
            plt.xlabel('Time (days)')
            plt.ylabel('$dYdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dYdt' + '.png')
            plt.show()
            plt.close()
        if i==5:
            plt.title('$dDdt$ versus time (days)')
            plt.plot(t, dDdt*population, label='dDdt', color='m')
            plt.xlabel('Time (days)')
            plt.ylabel('$dDdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dDdt' + '.png')
            plt.show()
            plt.close()
        if i==6:
            plt.title('$dQdt$ versus time (days)')
            plt.plot(t, dQdt*population, label='dQdt', color='m')
            plt.xlabel('Time (days)')
            plt.ylabel('$dQdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dQdt' + '.png')
            plt.show()
            plt.close()
        if i==7:
            plt.title('$dRdt$ versus time (days)')
            plt.plot(t, dRdt*population, label='dRdt', color='g')
            plt.xlabel('Time (days)')
            plt.ylabel('$dRdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dRdt' + '.png')
            plt.show()
            plt.close()
        if i==8:
            plt.title('$dFdt$ versus time (days)')
            plt.plot(t, dFdt*population, label='dFdt', color='k')
            plt.xlabel('Time (days)')
            plt.ylabel('$dFdt$')
            plt.legend()
            # plt.savefig('../Notebooks/figs/drums/derivatives/' + case_name + '_' + str(n_runs) + '_dFdt' + '.png')
            plt.show()
            plt.close()

In [28]:
# generate save path
mydir = os.path.join('../models/covasim/denoised', datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
os.makedirs(mydir)

In [29]:
# initialize model hyperparameters and architecture type
mask_input = True
eta_deep = True
beta_deep = True
tau_deep = False

In [30]:
# initialize model
binn = NNComponentsCV(params, 
                    u_tensor, 
                    t_max_real, 
                    tracing_array,
                    keep_d=keep_d, 
                    chi_type=chi_type,
                    mask_input=mask_input,
                    eta_deep=eta_deep,
                    beta_deep=beta_deep,
                    tau_deep=tau_deep)
binn.to(device)

NNComponentsCV(
  (eta_func): eta_NN(
    (mlp): BuildMLP(
      (activation): ReLU()
      (output_activation): Sigmoid()
      (MLP): Sequential(
        (0): Linear(in_features=4, out_features=256, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): ReLU()
        (5): Dropout(p=0.2, inplace=False)
        (6): Linear(in_features=256, out_features=256, bias=True)
        (7): ReLU()
        (8): Dropout(p=0.2, inplace=False)
        (9): Linear(in_features=256, out_features=1, bias=True)
        (10): Sigmoid()
      )
    )
  )
  (beta_func): beta_NN(
    (mlp): BuildMLP(
      (activation): ReLU()
      (output_activation): Sigmoid()
      (MLP): Sequential(
        (0): Linear(in_features=2, out_features=256, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): ReLU()
        (5): D

In [31]:
parameters = binn.parameters()
opt = torch.optim.Adam(parameters, lr=1e-4)
os.makedirs(os.path.join(mydir, case_name))
model = ModelWrapper(
    model=binn,
    optimizer=opt,
    loss=binn.loss,
    augmentation=None,
    # scheduler= scheduler,
    save_name=os.path.join(mydir, case_name) )
model.str_name = 'STEAYDQRF'

In [32]:
# save the range information before training
ranges = [binn.yita_lb, binn.yita_ub, binn.beta_lb, binn.beta_ub, binn.tau_lb, binn.tau_ub]
file_name = '_'.join([str(m) for m in ranges])
joblib.dump(None, os.path.join(mydir, file_name)) # model.save_folder
# if retrain
if retrain:
    model.load(model.save_name + '_best_val_model', device=device)
    model.model.train()
    model.save_name += '_retrain'
    
epochs = int(1000e3)
batch_size = 256
rel_save_thresh = 0.05

In [ ]:
# train jointly
model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=None,
    verbose=1,
    validation_data=[x_val, y_val],
    early_stopping=int(200e3),
    rel_save_thresh=rel_save_thresh)

Epoch 14134 | Train loss = 1.2234e-01 | Val loss = 1.1527e-01 | Remaining = 6:50:47                 

In [ ]:
total_train_losses = model.train_loss_list
total_val_losses = model.val_loss_list

plot_loss_convergence(total_train_losses, total_val_losses, rel_save_thresh, model.save_folder)